## get crop mask of the desired county and crop type of a specific year
and also get do corespondent ndvi map mask of that crop


In [ ]:
import ee
import geemap
import pandas as pd

from ras_to_vec import *

start = '2022-01-01'
end = '2022-01-14'

countyBoundary, winterWheatFC = ras_to_vec(20, 'Thomas', 2022)

# Load a Sentinel-2 image
s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(countyBoundary) \
    .filterDate(start, end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 100))\



# Get the first image in the collection
s2m = ee.Image(s2.mosaic())

# Calculate NDVI (Normalized Difference Vegetation Index)
ndvi = s2m.normalizedDifference(['B8', 'B4'])

# Clip the NDVI map with the winterWheatFC feature collection
clippedNDVI = ndvi.clipToCollection(winterWheatFC)

# Create a map and add the clipped NDVI map
Map = geemap.Map(basemap = 'SATELLITE')
Map.centerObject(winterWheatFC, 10)
palette = ['blue', 'green', 'white']
Map.addLayer(clippedNDVI, {'min': -1, 'max': 1, 'palette': 'Greens'}, 'Clipped NDVI')

# Style the county boundary to show only the borders
#Map.addLayer(countyBoundary, {'color': 'FF000000'}, 'County Boundary')
Map.addLayer(winterWheatFC, {'color': 'FF000000'}, 'Winter Wheat FC')

# Display the map
Map.addLayerControl()
Map

In [ ]:
# Get the NDVI values as a list
ndvi_list = clippedNDVI.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=winterWheatFC,
    scale=10,
).get('nd').getInfo()

# Print the NDVI values
print(ndvi_list)

In [ ]:
mean_ndvi = clippedNDVI.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=winterWheatFC,
    scale=30
).getInfo().get('nd')

mean_ndvi


In [ ]:
winterWheatFC.size()

In [ ]:
countyBoundary.size()

In [ ]:
import glob
import pandas as pd

data_path = 'data/'

csv_files = sorted(glob.glob(data_path + '*.csv'))

# Initialize an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Iterate over each CSV file
for csv_file in csv_files:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Append the DataFrame to the combined_df
    combined_df = combined_df.append(df, ignore_index=True)

# Print the combined DataFrame
combined_df


In [ ]:
import pandas as pd

df = pd.read_csv('data/loss_wheatwinter_countylevel_2000_2022.csv')
df = df[df['County'] != 'OTHER (COMBINED) COUNTIES']
# Reindex the DataFrame
df = df.reset_index(drop=True)
df

In [ ]:
df.to_csv('data/1loss_wheatwinter_countylevel_2000_2022.csv')

In [ ]:
import pandas as pd

df = pd.read_csv('data/usda_loss_wheatwinter_countylevel_2000_2022.csv')

df